# Hyperparameter Tuning
## Notebook Setup

In [8]:
import os

import optuna
import pandas as pd
import xgboost as xg
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
os.chdir(os.path.join("/", "home", "walkerdavis", "projects", "mpsc"))
DATA_ROOT = os.path.join("data", "processed")

## Dataset Loading and Splitting

In [3]:
df = pd.read_csv(os.path.join(DATA_ROOT, "Consolidated.csv"))

df = df.dropna(subset=["WeeklyWeightGainGPerKGPerDay"])

In [4]:
X = df.drop(
    columns=[
        "WeeklyWeightGainGPerDay",
        "WeeklyWeightGainGPerKGPerDay",
        "DailyWeightGainGPerDay",
        "DailyWeightGainGPerKGPerDay",
    ]
)
y = df["WeeklyWeightGainGPerKGPerDay"]

for col in X.select_dtypes(include=["object", "category"]).columns:
    X[col] = X[col].fillna("Missing")
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

for col in X.select_dtypes(include=["float64", "int64"]).columns:
    X[col] = X[col].fillna(X[col].median())

X = X.clip(lower=-1e6, upper=1e6)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [5]:
def model_results(preds, y_test):
    mae = mean_absolute_error(y_test, preds)
    mse = mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)

    print(f"MAE: {mae} | MSE: {mse} | r^2: {r2}")

## Quicktuning with Grid Search

In [9]:
param_grid = {
    'max_depth': [4, 5, 6, 7, 8],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 500, 1000]
}

model = xg.XGBRegressor(objective="reg:squarederror", n_estimators=100, seed=42)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

y_pred = grid_search.predict(X_test)

print(model_results(preds=y_pred, y_test=y_test))

MAE: 2.8433523196821837 | MSE: 54.10370784076627 | r^2: 0.5937654893693296
None


In [10]:
importances = pd.Series(grid_search.feature_importances_, index=X_train.columns).sort_values(ascending=False)
importances.head(15)

AttributeError: 'GridSearchCV' object has no attribute 'feature_importances_'

## Hyperparam Tuning with Optuna

In [22]:
def objective_func(trial: optuna.Trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 1000)
    max_depth = trial.suggest_int("max_depth", 10, 50)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 32)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0, 1)
    objective = trial.suggest_categorical(
        "objective",
        [
            "reg:squarederror",
            "reg:absoluteerror",
        ],
    )
    learning_rate = trial.suggest_float("learning_rate", 0.005, 0.25)

    model = xg.XGBRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_child_weight=min_child_weight,
        colsample_bytree=colsample_bytree,
        objective=objective,
        learning_rate=learning_rate,
        random_state=42,
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    score = r2_score(y_test, y_pred)

    return score


study = optuna.create_study(
    direction="maximize", sampler=optuna.samplers.RandomSampler(seed=42)
)
study.optimize(objective_func, n_trials=1000)

print("Best Trial")
trial = study.best_trial

print(f"Value: {trial.value}")

print("Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-07-17 08:44:09,904] A new study created in memory with name: no-name-fa29308d-a8d2-449c-9635-325443de369a
[I 2025-07-17 08:44:24,868] Trial 0 finished with value: 0.4932490813725503 and parameters: {'n_estimators': 437, 'max_depth': 48, 'min_child_weight': 24, 'colsample_bytree': 0.5986584841970366, 'objective': 'reg:squarederror', 'learning_rate': 0.019230484981208867}. Best is trial 0 with value: 0.4932490813725503.
[I 2025-07-17 08:44:42,763] Trial 1 finished with value: 0.3833568090115923 and parameters: {'n_estimators': 880, 'max_depth': 34, 'min_child_weight': 23, 'colsample_bytree': 0.020584494295802447, 'objective': 'reg:squarederror', 'learning_rate': 0.057023082116177654}. Best is trial 0 with value: 0.4932490813725503.
[I 2025-07-17 08:44:49,372] Trial 2 finished with value: 0.5269831450152953 and parameters: {'n_estimators': 263, 'max_depth': 17, 'min_child_weight': 10, 'colsample_bytree': 0.5247564316322378, 'objective': 'reg:squarederror', 'learning_rate': 0.15490

Best Trial
Value: 0.637264985933534
Params:
    n_estimators: 310
    max_depth: 11
    min_child_weight: 1
    colsample_bytree: 0.9877223897360315
    objective: reg:squarederror
    learning_rate: 0.17151358425980212


In [ ]:
best_params = study.best_params

best_n_estimators = best_params["n_estimators"]
best_max_depth = best_params["max_depth"]
best_min_child_weight = best_params["min_child_weight"]
best_colsample_bytree = best_params["colsample_bytree"]
best_objective = best_params["objective"]
best_learning_rate = best_params["learning_rate"]

best_model = xg.XGBRegressor(
    n_estimators=best_n_estimators,
    max_depth=best_max_depth,
    min_child_weight=best_min_child_weight,
    colsample_bytree=best_colsample_bytree,
    objective=best_objective,
    learning_rate=best_learning_rate,
)

best_model.fit(X_train, y_train)

y_pred_xgb_optuna_200_trials = best_model.predict(X_test)

model_results(y_pred_xgb_optuna_200_trials, y_test)

MAE: 2.831387331460588 | MSE: 48.71796382861781 | r^2: 0.6107241278870943
